# ETL

Firstly, we have to import the right libraries

In [3]:
import pandas as pd
import numpy as np

We download the file `LA Crime_Data_from_2010_to_2019.csv` from [Kaggle](https://www.kaggle.com/jpmiller/police-violence-racial-equity) and move it to the same folder as this notebook
* Reading the csv file

In [4]:
la_crimes = pd.read_csv('LA Crime_Data_from_2010_to_2019.csv')
la_crimes.rename(columns={'AREA ': 'AREA'}, inplace = True)
la_crimes

DR_NO               Date Rptd                DATE OCC  TIME OCC  \
0          1307355  02/20/2010 12:00:00 AM  02/20/2010 12:00:00 AM      1350   
1         11401303  09/13/2010 12:00:00 AM  09/12/2010 12:00:00 AM        45   
2         70309629  08/09/2010 12:00:00 AM  08/09/2010 12:00:00 AM      1515   
3         90631215  01/05/2010 12:00:00 AM  01/05/2010 12:00:00 AM       150   
4        100100501  01/03/2010 12:00:00 AM  01/02/2010 12:00:00 AM      2100   
...            ...                     ...                     ...       ...   
2114694  190608903  03/28/2019 12:00:00 AM  03/28/2019 12:00:00 AM       400   
2114695  190715222  08/15/2019 12:00:00 AM  08/14/2019 12:00:00 AM      1810   
2114696  192004409  01/06/2019 12:00:00 AM  01/06/2019 12:00:00 AM      2100   
2114697  191716777  10/17/2019 12:00:00 AM  10/16/2019 12:00:00 AM      1800   
2114698  190805435  02/01/2019 12:00:00 AM  02/01/2019 12:00:00 AM      1615   

         AREA   AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0          13      Newton         1385         2     900   
1          14     Pacific         1485         2     740   
2          13      Newton         1324         2     946   
3           6   Hollywood          646         2     900   
4           1     Central          176         1     122   
...       ...         ...          ...       ...     ...   
2114694     6   Hollywood          644         1     648   
2114695     7    Wilshire          701         1     331   
2114696    20     Olympic         2029         2     930   
2114697    17  Devonshire         1795         1     420   
2114698     8     West LA          852         1     330   

                                               Crm Cd Desc  ... Status  \
0                                 VIOLATION OF COURT ORDER  ...     AA   
1        VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...  ...     IC   
2                                OTHER MISCELLANEOUS CRIME  ...     IC   
3                                 VIOLATION OF COURT ORDER  ...     IC   
4                                          RAPE, ATTEMPTED  ...     IC   
...                                                    ...  ...    ...   
2114694                                              ARSON  ...     IC   
2114695   THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)  ...     IC   
2114696             CRIMINAL THREATS - NO WEAPON DISPLAYED  ...     IC   
2114697    THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)  ...     IC   
2114698                              BURGLARY FROM VEHICLE  ...     IC   

          Status Desc Crm Cd 1 Crm Cd 2  Crm Cd 3 Crm Cd 4  \
0        Adult Arrest    900.0      NaN       NaN      NaN   
1         Invest Cont    740.0      NaN       NaN      NaN   
2         Invest Cont    946.0      NaN       NaN      NaN   
3         Invest Cont    900.0    998.0       NaN      NaN   
4         Invest Cont    122.0      NaN       NaN      NaN   
...               ...      ...      ...       ...      ...   
2114694   Invest Cont    648.0      NaN       NaN      NaN   
2114695   Invest Cont    331.0      NaN       NaN      NaN   
2114696   Invest Cont    930.0      NaN       NaN      NaN   
2114697   Invest Cont    420.0      NaN       NaN      NaN   
2114698   Invest Cont    330.0      NaN       NaN      NaN   

                                         LOCATION  \
0          300 E  GAGE                         AV   
1                 SEPULVEDA                    BL   
2         1300 E  21ST                         ST   
3                 CAHUENGA                     BL   
4                 8TH                          ST   
...                                           ...   
2114694   1400 N  LA BREA                      AV   
2114695           WILLOUGHBY                   AV   
2114696                                       6TH   
2114697  17200    NAPA                         ST   
2114698   1700    BARRY                        AV   

                            Cross Street      LAT       LON 

## DR_NO column
* We see that the column `DR_NO` is a unique code for each row, that means each crime reported has a different code

In [5]:
la_crimes['DR_NO'].value_counts()

151001085    1
161811979    1
111721424    1
121320908    1
121316814    1
            ..
130712581    1
130718726    1
151714826    1
151712779    1
151001091    1
Name: DR_NO, Length: 2114699, dtype: int64

## Date Rptd and DATE OCC columns
* We see that all the dates have the time of 12:00:00 AM

In [6]:
la_crimes['Date Rptd'].str[-8:].value_counts() #selecting only the time and getting the different values of it

00:00 AM    2114699
Name: Date Rptd, dtype: int64

In [7]:
la_crimes['DATE OCC'].str[-8:].value_counts() #selecting only the time and getting the different values of it

00:00 AM    2114699
Name: DATE OCC, dtype: int64

* As result, we can delete the data of the time because it does not give additional information to us

In [8]:
la_crimes['Date Rptd'] = la_crimes['Date Rptd'].str[:10] #only keeping the data of the date without the time

In [9]:
la_crimes['DATE OCC'] = la_crimes['DATE OCC'].str[:10] #only keeping the data

* Now we are going to bring the date format to the appropriate one for databases: YYYY-MM-DD

In [10]:
la_crimes['Date Rptd'] = pd.to_datetime(la_crimes['Date Rptd'])
la_crimes['DATE OCC'] = pd.to_datetime(la_crimes['DATE OCC'])
la_crimes['Date Rptd'].sample(5)

1441765   2017-05-13
1410879   2016-02-05
896600    2014-08-04
36846     2010-12-05
471457    2014-06-17
Name: Date Rptd, dtype: datetime64[ns]

## TIME OCC column
* Now we have to fix the column  `TIME OCC` to a '24hour:minutes' format
* First we transform the column to datatype string because it was integer
* Then we add  '00' in front of every date (because the rows with time occured at 12AM have no values for hours
* Lastly we only select the right string characters

In [11]:
la_crimes['TIME OCC'] = la_crimes['TIME OCC'].astype(str)

In [12]:
la_crimes['TIME OCC']= '00'+la_crimes['TIME OCC']

In [13]:
la_crimes['TIME OCC'] = la_crimes['TIME OCC'].str[-4:-2] + ':' +la_crimes['TIME OCC'].str[-2:]

* We join the  columns `DATE OCC` and `TIME OCC` into the column `DATE OCC` so that it will be in the format YYYY-MM-DD hh:mm. It will be saved as datatype datetime YYYY-MM-DD hh:mm:ss in the database
* We drop the column `TIME OCC`

In [14]:
la_crimes['DATE OCC'] = la_crimes['DATE OCC'].astype(str) +' ' + la_crimes['TIME OCC']
la_crimes['DATE OCC'] = pd.to_datetime(la_crimes['DATE OCC'])

In [15]:
la_crimes = la_crimes.drop(['TIME OCC'], axis=1)

## Mocodes column
We see that some crime reports have null values for the column of `Mocodes`
* We replace those values with the value `unknown`

In [16]:
la_crimes.loc[la_crimes['Mocodes'].isna()]

DR_NO  Date Rptd            DATE OCC  AREA   AREA NAME  \
15       100100535 2010-01-17 2010-01-16 17:35:00     1     Central   
28       100100578 2010-02-05 2010-02-03 12:55:00     1     Central   
51       100100654 2010-02-27 2010-02-27 19:55:00     1     Central   
79       100100730 2010-03-23 2010-03-20 12:15:00     1     Central   
102      100100786 2010-04-08 2010-04-08 02:20:00     1     Central   
...            ...        ...                 ...   ...         ...   
2114663  190417846 2019-12-11 2019-12-06 18:00:00     4  Hollenbeck   
2114670  191710450 2019-05-25 2019-05-25 06:30:00    17  Devonshire   
2114683  191816545 2019-07-22 2019-07-17 20:30:00    18   Southeast   
2114691  191307168 2019-02-28 2019-02-28 07:00:00    13      Newton   
2114697  191716777 2019-10-17 2019-10-16 18:00:00    17  Devonshire   

         Rpt Dist No  Part 1-2  Crm Cd  \
15               185         2     946   
28               185         2     946   
51               174         2     946   
79               111         2     647   
102              161         1     510   
...              ...       ...     ...   
2114663          423         1     510   
2114670         1761         1     510   
2114683         1836         2     922   
2114691         1394         1     510   
2114697         1795         1     420   

                                             Crm Cd Desc Mocodes  ...  Status  \
15                             OTHER MISCELLANEOUS CRIME     NaN  ...      IC   
28                             OTHER MISCELLANEOUS CRIME     NaN  ...      IC   
51                             OTHER MISCELLANEOUS CRIME     NaN  ...      AA   
79                     THROWING OBJECT AT MOVING VEHICLE     NaN  ...      IC   
102                                     VEHICLE - STOLEN     NaN  ...      IC   
...                                                  ...     ...  ...     ...   
2114663                                 VEHICLE - STOLEN     NaN  ...      IC   
2114670                                 VEHICLE - STOLEN     NaN  ...      IC   
2114683                                   CHILD STEALING     NaN  ...      AO   
2114691                                 VEHICLE - STOLEN     NaN  ...      IC   
2114697  THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)     NaN  ...      IC   

          Status Desc Crm Cd 1  Crm Cd 2 Crm Cd 3  Crm Cd 4  \
15        Invest Cont    946.0     999.0      NaN       NaN   
28        Invest Cont    946.0     999.0      NaN       NaN   
51       Adult Arrest    946.0       NaN      NaN       NaN   
79        Invest Cont    647.0       NaN      NaN       NaN   
102       Invest Cont    510.0     520.0      NaN       NaN   
...               ...      ...       ...      ...       ...   
2114663   Invest Cont    510.0       NaN      NaN       NaN   
2114670   Invest Cont    510.0       NaN      NaN       NaN   
2114683   Adult Other    922.0       NaN      NaN       NaN   
2114691   Invest Cont    510.0       NaN      NaN       NaN   
2114697   Invest Cont    420.0       NaN      NaN       NaN   

                                         LOCATION  \
15         300 E  OLYMPIC                      BL   
28        1200    MAPLE                        AV   
51             W  7TH                          ST   
79                                 CESAR E CHAVEZ   
102               FRANCISCO                    ST   
...                                           ...   
2114663   1100 N  MISSION                      RD   
2114670   9200    ETON                         AV   
2114683  10400    FIRTH                        AV   
2114691    100 E  67TH                         ST   
2114697  17200    NAPA                         ST   

                               Cross Street      LAT       LON  
15                                      NaN  34.0389 -118.2550  
28                                      NaN  34.0357 -118.2563  
51       S  SPRING                       ST  34.0445 -118.2523  
79          FIGUEROA        

In [17]:
indexes_null_mo = la_crimes.loc[la_crimes ['Mocodes'].isna()].index
la_crimes.loc[indexes_null_mo,'Mocodes'] = 'unknown'

## Vict Age column
* Now we are going to fix the column `Vict Age` that don't make any sense
* Specifically we are going to change all negative age numbers to `0` which we consider missing value.

In [18]:
la_crimes['Vict Age'].value_counts()

 0      369886
 25      48101
 26      47469
 27      47011
 24      46739
         ...  
-7          15
-8           7
-9           4
 114         1
 118         1
Name: Vict Age, Length: 110, dtype: int64

In [19]:
indexes_neg_age = la_crimes.loc[la_crimes['Vict Age'] < 0]['Vict Age'].index #finding the indexes of negative ages
la_crimes.loc[indexes_neg_age,'Vict Age'] = 0

## Vict Sex column
* Now we fix the column `Vict Sex` which is the victim's sex.
* According to the description the values are: F - Female, M - Male, X - Unknown

In [20]:
la_crimes['Vict Sex'].value_counts()

M    974309
F    888499
X     55129
H        73
N        17
-         1
Name: Vict Sex, dtype: int64

* We see there are a bunch of irrelevant values which we change to 'X'

In [21]:
indexes_wrong_sex = la_crimes.loc[(la_crimes['Vict Sex'] != 'M') & (la_crimes['Vict Sex'] != 'F')].index
la_crimes.loc[indexes_wrong_sex,'Vict Sex'] = 'X'

## Vict Descent column
Now we will fix the column `Vict Descent`. We see there are a bunch of nan values. Also, according to the description of the dataset the different descents are: : 
* A - Other Asian 
* B - Black 
* C - Chinese  
* D - Cambodian 
* F - Filipino 
* G - Guamanian 
* H - Hispanic/Latin/Mexican 
* I - American Indian/Alaskan Native 
* J - Japanese 
* K - Korean 
* L - Laotian 
* O - Other 
* P - Pacific Islander 
* S - Samoan 
* U - Hawaiian 
* V - Vietnamese 
* W - White 
* X - Unknown 
* Z - Asian Indian

In [22]:
print("The reported crimes with Nan (null) values for the column Vict Descent are:")
len(la_crimes.loc[la_crimes['Vict Descent'].isna()])

The reported crimes with Nan (null) values for the column Vict Descent are:


196718

In [23]:
la_crimes['Vict Descent'].value_counts()

H    725348
W    510158
B    335102
O    202969
X     78147
A     51109
K      9141
F      2553
C      1061
I       945
J       418
P       343
V       201
U       190
Z       136
G        85
S        31
D        23
L        18
-         3
Name: Vict Descent, dtype: int64

In [24]:
indexes_wrong_decent = la_crimes.loc[(la_crimes['Vict Descent'] == '-') | (la_crimes['Vict Descent'].isna())].index
la_crimes.loc[indexes_wrong_decent,'Vict Descent'] = 'X'

## Premis Cd and Premis Desc columns
Now we see that many rows have null values for the column `Premis Cd` 
* Because this column has float values, we replace the null values with `0`

In [25]:
la_crimes.loc[la_crimes['Premis Cd'].isna()]

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
6590     100121447 2010-12-12 2010-12-12 11:50:00     1      Central   
32148    100913648 2010-06-21 2010-06-20 14:35:00     9     Van Nuys   
67340    100816222 2010-09-03 2010-04-16 00:01:00     8      West LA   
68276    100818222 2010-11-18 2010-10-16 16:00:00     8      West LA   
71523    100908076 2010-03-16 2010-03-15 22:05:00    15  N Hollywood   
85649    101017796 2010-09-21 2010-09-08 17:20:00    10  West Valley   
92668    101114489 2010-06-18 2010-06-18 19:10:00    11    Northeast   
102768   101214269 2010-05-13 2010-05-12 18:00:00    12  77th Street   
107695   101223939 2010-09-17 2010-06-01 12:00:00    12  77th Street   
124293   101410405 2010-04-12 2010-04-11 23:20:00    14      Pacific   
181888   101926208 2010-12-07 2010-12-07 13:30:00    19      Mission   
223337   111800860 2011-04-25 2011-04-24 20:30:00    18    Southeast   
236205   111614486 2011-08-07 2011-08-07 12:20:00    16     Foothill   
244481   110104233 2011-01-06 2011-01-02 16:19:00     1      Central   
246857   111820353 2011-10-21 2011-09-13 10:02:00    18    Southeast   
256518   111412807 2011-04-16 2011-04-16 16:00:00    14      Pacific   
289994   112020223 2011-10-20 2011-10-13 20:00:00    20      Olympic   
326213   110616246 2011-06-20 2011-06-20 04:10:00     6    Hollywood   
327485   111400001 2011-02-11 2011-02-10 20:45:00    14      Pacific   
390593   111428080 2011-10-05 2011-10-05 14:05:00    14      Pacific   
441754   121815392 2012-08-06 2012-08-04 17:30:00    18    Southeast   
530688   120706377 2012-02-23 2012-02-23 17:30:00     7     Wilshire   
563146   120619583 2012-07-16 2012-06-16 12:00:00     6    Hollywood   
570068   121412515 2012-05-03 2012-05-03 15:20:00    14      Pacific   
603839   121801492 2012-11-22 2012-11-21 23:00:00    18    Southeast   
684026   131416539 2013-06-24 2013-06-24 01:05:00    14      Pacific   
691500   131706326 2013-02-20 2013-02-20 17:50:00    17   Devonshire   
794923   131511839 2013-05-29 2013-05-29 08:20:00    15  N Hollywood   
811749   151704507 2015-01-13 2014-01-13 13:30:00    17   Devonshire   
827479   141817287 2014-09-17 2014-09-16 08:11:00    18    Southeast   
878694   140511089 2014-06-21 2014-06-21 14:35:00     5       Harbor   
888806   142111719 2014-06-21 2014-06-21 21:12:00    21      Topanga   
915055   140913407 2014-07-18 2014-07-18 10:00:00     9     Van Nuys   
950356   141012475 2014-07-19 2014-07-19 20:40:00    10  West Valley   
957521   141811155 2014-05-24 2014-05-22 18:50:00    18    Southeast   
967557   150206736 2015-02-20 2014-11-18 23:00:00     2      Rampart   
967979   141000807 2014-07-19 2014-07-19 10:30:00    10  West Valley   
977511   142111721 2014-06-22 2014-06-21 22:00:00    21      Topanga   
987563   141508150 2014-03-23 2014-03-23 23:30:00    15  N Hollywood   
1294312  161112881 2016-07-04 2016-07-04 00:05:00    11    Northeast   
1364942  170906911 2017-02-27 2016-11-24 17:00:00     9     Van Nuys   
1417580  161400604 2016-04-17 2016-04-14 12:00:00    10  West Valley   
1452779  171704122 2017-01-03 2017-01-02 18:00:00    17   Devonshire   
1465775  171804995 2017-01-18 2017-01-18 18:45:00    18    Southeast   
1511359  171804978 2017-01-18 2017-01-18 14:48:00    18    Southeast   
1667060  170918599 2017-10-01 2017-09-30 19:00:00     9     Van Nuys   
1814056  180804421 2018-01-11 2018-01-10 23:00:00     8      West LA   
1818531  181719036 2018-11-17 2018-09-11 12:30:00    17   Devonshire   
1947458  191307542 2019-03-06 2019-03-06 16:45:00    13       Newton   
1968622  201604049 2020-01-02 2019-11-20 15:30:00    16     Foothill   
1975814  201604048 2020-01-02 2019-11-20 15:30:00    16     Foothill   
2030344  191204256 2019-01-05 2019-01-04 23:30:00    12  77th Street   
2102344  190616200 2019-07-26 2019-07-26 04:20:00     6    Hollywood   
2108045  190408173 2019-04-06 2019-04-06 06:00:00     4   Hollenbeck   

         Rpt Dist No  Part 1-2  Crm Cd  \
6590      

In [26]:
la_crimes.loc[la_crimes['Premis Desc'] =='unknown'][['Premis Cd','Premis Desc']]

Empty DataFrame
Columns: [Premis Cd, Premis Desc]
Index: []

In [27]:
indexes_null_premiscd = la_crimes.loc[la_crimes ['Premis Cd'].isna()].index
la_crimes.loc[indexes_null_premiscd,'Premis Cd'] = 0

Now we see that many rows have null values for the column `Premis Desc`

* We replace those null values with `unknown`

In [28]:
la_crimes.loc[la_crimes['Premis Desc'].isna()]

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
6590     100121447 2010-12-12 2010-12-12 11:50:00     1      Central   
32148    100913648 2010-06-21 2010-06-20 14:35:00     9     Van Nuys   
67340    100816222 2010-09-03 2010-04-16 00:01:00     8      West LA   
68276    100818222 2010-11-18 2010-10-16 16:00:00     8      West LA   
71523    100908076 2010-03-16 2010-03-15 22:05:00    15  N Hollywood   
...            ...        ...                 ...   ...          ...   
2109009  190223459 2019-12-17 2019-12-15 21:45:00     2      Rampart   
2109581  191515929 2019-08-20 2019-08-20 05:40:00    15  N Hollywood   
2113695  190119416 2019-07-28 2019-07-27 18:00:00     1      Central   
2114205  191221441 2019-08-29 2019-08-29 01:00:00    12  77th Street   
2114388  190616877 2019-08-07 2019-08-07 03:14:00     6    Hollywood   

         Rpt Dist No  Part 1-2  Crm Cd  \
6590             185         1     110   
32148            915         1     510   
67340            803         2     813   
68276            811         2     812   
71523           1547         1     420   
...              ...       ...     ...   
2109009          236         2     668   
2109581         1599         1     310   
2113695          157         1     320   
2114205         1268         1     110   
2114388          649         1     310   

                                               Crm Cd Desc  \
6590                                     CRIMINAL HOMICIDE   
32148                                     VEHICLE - STOLEN   
67340                       CHILD ANNOYING (17YRS & UNDER)   
68276    CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...   
71523      THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
...                                                    ...   
2109009         EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)   
2109581                                           BURGLARY   
2113695                                BURGLARY, ATTEMPTED   
2114205                                  CRIMINAL HOMICIDE   
2114388                                           BURGLARY   

                                    Mocodes  ...  Status   Status Desc  \
6590                                unknown  ...      AA  Adult Arrest   
32148                               unknown  ...      IC   Invest Cont   
67340                               unknown  ...      IC   Invest Cont   
68276                               unknown  ...      IC   Invest Cont   
71523                               unknown  ...      IC   Invest Cont   
...                                     ...  ...     ...           ...   
2109009            0344 1309 1822 0913 1803  ...      IC   Invest Cont   
2109581  2018 0344 1309 1609 1402 1414 0384  ...      IC   Invest Cont   
2113695                                1607  ...      IC   Invest Cont   
2114205            1100 0430 1402 1822 0906  ...      AA  Adult Arrest   
2114388                      1607 0344 1402  ...      IC   Invest Cont   

        Crm Cd 1  Crm Cd 2 Crm Cd 3  Crm Cd 4  \
6590       110.0       NaN      NaN       NaN   
32148      510.0       NaN      NaN       NaN   
67340      813.0       NaN      NaN       NaN   
68276      812.0       NaN      NaN       NaN   
71523      420.0       NaN      NaN       NaN   
...          ...       ...      ...       ...   
2109009    668.0       NaN      NaN       NaN   
2109581    310.0     998.0      NaN       NaN   
2113695    320.0       NaN      NaN       NaN   
2114205    110.0     998.0      NaN       NaN   
2114388    310.0       NaN      NaN       NaN   

                                        LOCATION Cross Street      LAT  \
6590      200 W  OLYMPIC                      BL          NaN  34.0409   
32148    7000    VAN NUYS                     BL          NaN  34.1976   
67340                       2000    MANDAVILLE C          NaN  34.0949   
68276    1100    LAS PULGAS                   RD          NaN  34.0528   
71523                                       BECK  

In [29]:
indexes_null_premisdesc = la_crimes.loc[la_crimes ['Premis Desc'].isna()].index
la_crimes.loc[indexes_null_premisdesc,'Premis Desc'] = 'unknown'

Theoretically, for the rows with `Premis Desc` = 'unknown', the column `Premis Cd` should be equal to $0$ (unknown). However, there are some rows with `Premis Cd` different than 0

In [30]:
la_crimes.loc[la_crimes['Premis Desc']=='unknown'][['Premis Desc','Premis Cd']]

Premis Desc  Premis Cd
6590        unknown        0.0
32148       unknown        0.0
67340       unknown        0.0
68276       unknown        0.0
71523       unknown        0.0
...             ...        ...
2109009     unknown      418.0
2109581     unknown      256.0
2113695     unknown      256.0
2114205     unknown      256.0
2114388     unknown      418.0

[188 rows x 2 columns]

In detail, the rows with `Premis Desc` = 'unknown', have the values of $0$ or $256$ or $418$ or $838$

In [31]:
la_crimes.loc[la_crimes['Premis Desc']=='unknown']['Premis Cd'].unique()

array([  0., 838., 418., 256.])

* Next up we see that all the rows with `Premis Cd` = $418$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [32]:
cd418 = la_crimes.loc[la_crimes['Premis Cd']==418][['Premis Desc','Premis Cd']]
cd418['Premis Desc'].unique()

array(['unknown'], dtype=object)

* Next up we see that all the rows with `Premis Cd` = $256$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [33]:
cd256 = la_crimes.loc[la_crimes['Premis Cd']== 256][['Premis Desc','Premis Cd']]
cd256['Premis Desc'].unique()

array(['unknown'], dtype=object)

* Next up we see that all the rows with `Premis Cd` = $838$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [34]:
cd838 = la_crimes.loc[la_crimes['Premis Cd']== 838][['Premis Desc','Premis Cd']]
cd838['Premis Desc'].unique()

array(['unknown'], dtype=object)

* We change all the rows with `Premis Cd` equal to $256$, $418$ or $838$, to $0$ (unknown)

In [35]:
indexes_wrong_premiscd = la_crimes.loc[(la_crimes ['Premis Cd'] == 256) | 
                                      (la_crimes ['Premis Cd'] == 418) |
                                      (la_crimes ['Premis Cd'] == 838)].index
la_crimes.loc[indexes_wrong_premiscd,'Premis Cd'] = 0

* Lastly, we transform `Premis Cd` from float to integer

In [36]:
la_crimes['Premis Cd'] = la_crimes['Premis Cd'].astype(int)

## Weapon Used Cd and Weapon Desc columns
Some rows have null values for the column `Weapon Used Cd`

* We replace those null values with `0` because this column contains float datatypes

In [37]:
la_crimes.loc[la_crimes['Weapon Used Cd'].isna()]

DR_NO  Date Rptd            DATE OCC  AREA   AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13      Newton   
1         11401303 2010-09-13 2010-09-12 00:45:00    14     Pacific   
2         70309629 2010-08-09 2010-08-09 15:15:00    13      Newton   
5        100100506 2010-01-05 2010-01-04 16:50:00     1     Central   
6        100100508 2010-01-08 2010-01-07 20:05:00     1     Central   
...            ...        ...                 ...   ...         ...   
2114687  191408297 2019-03-11 2019-03-08 12:00:00    14     Pacific   
2114691  191307168 2019-02-28 2019-02-28 07:00:00    13      Newton   
2114695  190715222 2019-08-15 2019-08-14 18:10:00     7    Wilshire   
2114697  191716777 2019-10-17 2019-10-16 18:00:00    17  Devonshire   
2114698  190805435 2019-02-01 2019-02-01 16:15:00     8     West LA   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1485         2     740   
2               1324         2     946   
5                162         1     442   
6                182         1     330   
...              ...       ...     ...   
2114687         1438         1     440   
2114691         1394         1     510   
2114695          701         1     331   
2114697         1795         1     420   
2114698          852         1     330   

                                               Crm Cd Desc  \
0                                 VIOLATION OF COURT ORDER   
1        VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...   
2                                OTHER MISCELLANEOUS CRIME   
5                 SHOPLIFTING - PETTY THEFT ($950 & UNDER)   
6                                    BURGLARY FROM VEHICLE   
...                                                    ...   
2114687                 THEFT PLAIN - PETTY ($950 & UNDER)   
2114691                                   VEHICLE - STOLEN   
2114695   THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)   
2114697    THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
2114698                              BURGLARY FROM VEHICLE   

                                    Mocodes  ...  Status   Status Desc  \
0                            0913 1814 2000  ...      AA  Adult Arrest   
1                                      0329  ...      IC   Invest Cont   
2                                      0344  ...      IC   Invest Cont   
5                                 0344 1402  ...      AA  Adult Arrest   
6                                      0344  ...      IC   Invest Cont   
...                                     ...  ...     ...           ...   
2114687            0344 1501 1608 1607 0349  ...      IC   Invest Cont   
2114691                             unknown  ...      IC   Invest Cont   
2114695                           1300 0344  ...      IC   Invest Cont   
2114697                             unknown  ...      IC   Invest Cont   
2114698  1302 1609 0358 1307 0344 0377 0321  ...      IC   Invest Cont   

        Crm Cd 1  Crm Cd 2 Crm Cd 3  Crm Cd 4  \
0          900.0       NaN      NaN       NaN   
1          740.0       NaN      NaN       NaN   
2          946.0       NaN      NaN       NaN   
5          442.0       NaN      NaN       NaN   
6          330.0       NaN      NaN       NaN   
...          ...       ...      ...       ...   
2114687    440.0       NaN      NaN       NaN   
2114691    510.0       NaN      NaN       NaN   
2114695    331.0       NaN      NaN       NaN   
2114697    420.0       NaN      NaN       NaN   
2114698    330.0       NaN      NaN       NaN   

                                         LOCATION  \
0          300 E  GAGE                         AV   
1                 SEPULVEDA                    BL   
2         1300 E  21ST                         ST   
5          700 W  7TH                          ST   
6                 PICO                         BL   
...                                           ...   
2114687   3600    MIDVALE                      AV   
2114691 

In [38]:
indexes_null_weaponcd = la_crimes.loc[la_crimes ['Weapon Used Cd'].isna()].index
la_crimes.loc[indexes_null_weaponcd,'Weapon Used Cd'] = 0

Also rows have null values for the column `Weapon Desc`

* We replace those null values with `unknown` because this column contains object datatypes

In [39]:
la_crimes.loc[la_crimes['Weapon Desc'].isna()]

DR_NO  Date Rptd            DATE OCC  AREA   AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13      Newton   
1         11401303 2010-09-13 2010-09-12 00:45:00    14     Pacific   
2         70309629 2010-08-09 2010-08-09 15:15:00    13      Newton   
5        100100506 2010-01-05 2010-01-04 16:50:00     1     Central   
6        100100508 2010-01-08 2010-01-07 20:05:00     1     Central   
...            ...        ...                 ...   ...         ...   
2114687  191408297 2019-03-11 2019-03-08 12:00:00    14     Pacific   
2114691  191307168 2019-02-28 2019-02-28 07:00:00    13      Newton   
2114695  190715222 2019-08-15 2019-08-14 18:10:00     7    Wilshire   
2114697  191716777 2019-10-17 2019-10-16 18:00:00    17  Devonshire   
2114698  190805435 2019-02-01 2019-02-01 16:15:00     8     West LA   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1485         2     740   
2               1324         2     946   
5                162         1     442   
6                182         1     330   
...              ...       ...     ...   
2114687         1438         1     440   
2114691         1394         1     510   
2114695          701         1     331   
2114697         1795         1     420   
2114698          852         1     330   

                                               Crm Cd Desc  \
0                                 VIOLATION OF COURT ORDER   
1        VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...   
2                                OTHER MISCELLANEOUS CRIME   
5                 SHOPLIFTING - PETTY THEFT ($950 & UNDER)   
6                                    BURGLARY FROM VEHICLE   
...                                                    ...   
2114687                 THEFT PLAIN - PETTY ($950 & UNDER)   
2114691                                   VEHICLE - STOLEN   
2114695   THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)   
2114697    THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
2114698                              BURGLARY FROM VEHICLE   

                                    Mocodes  ...  Status   Status Desc  \
0                            0913 1814 2000  ...      AA  Adult Arrest   
1                                      0329  ...      IC   Invest Cont   
2                                      0344  ...      IC   Invest Cont   
5                                 0344 1402  ...      AA  Adult Arrest   
6                                      0344  ...      IC   Invest Cont   
...                                     ...  ...     ...           ...   
2114687            0344 1501 1608 1607 0349  ...      IC   Invest Cont   
2114691                             unknown  ...      IC   Invest Cont   
2114695                           1300 0344  ...      IC   Invest Cont   
2114697                             unknown  ...      IC   Invest Cont   
2114698  1302 1609 0358 1307 0344 0377 0321  ...      IC   Invest Cont   

        Crm Cd 1  Crm Cd 2 Crm Cd 3  Crm Cd 4  \
0          900.0       NaN      NaN       NaN   
1          740.0       NaN      NaN       NaN   
2          946.0       NaN      NaN       NaN   
5          442.0       NaN      NaN       NaN   
6          330.0       NaN      NaN       NaN   
...          ...       ...      ...       ...   
2114687    440.0       NaN      NaN       NaN   
2114691    510.0       NaN      NaN       NaN   
2114695    331.0       NaN      NaN       NaN   
2114697    420.0       NaN      NaN       NaN   
2114698    330.0       NaN      NaN       NaN   

                                         LOCATION  \
0          300 E  GAGE                         AV   
1                 SEPULVEDA                    BL   
2         1300 E  21ST                         ST   
5          700 W  7TH                          ST   
6                 PICO                         BL   
...                                           ...   
2114687   3600    MIDVALE                      AV   
2114691 

In [40]:
indexes_null_weapondesc = la_crimes.loc[la_crimes ['Weapon Desc'].isna()].index
la_crimes.loc[indexes_null_weapondesc,'Weapon Desc'] = 'unknown'

Lastly, we transform the column `Weapon Used Cd` from float to integer

In [41]:
 la_crimes['Weapon Used Cd'] = la_crimes['Weapon Used Cd'].astype(int)

## Status and Status Desc columns
* Now we check the status of the crime incident
* We see that the columns `Status` and `Status Desc` have some differences.

In [42]:
la_crimes['Status'].value_counts()

IC    1623829
AO     250589
AA     219081
JA      15864
JO       5301
CC         29
19          1
TH          1
13          1
Name: Status, dtype: int64

In [43]:
la_crimes['Status Desc'].value_counts()

Invest Cont     1623829
Adult Other      250589
Adult Arrest     219081
Juv Arrest        15864
Juv Other          5301
UNK                  35
Name: Status Desc, dtype: int64

* We observe that all the values with a Status `CC` , `19` , `TH` , `13` have a Status Description `UNK`
* That's why we change the rows with a Status `19` , `TH` , `13`(because those only appear once) or null to `CC` 

In [44]:
la_crimes.loc[la_crimes['Status Desc'] == 'UNK']

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
100040   101208618 2010-03-02 2010-03-02 03:30:00    12  77th Street   
151803   101700682 2010-03-09 2010-03-08 11:55:00    17   Devonshire   
160732   101721148 2010-11-15 2010-11-14 17:00:00    17   Devonshire   
219776   112109831 2011-04-29 2011-04-28 22:00:00    21      Topanga   
285526   111204921 2011-01-14 2011-01-12 18:30:00    12  77th Street   
400260   111225404 2011-10-26 2011-10-26 07:20:00    12  77th Street   
485891   141215426 2014-06-30 2012-01-01 12:00:00    12  77th Street   
563146   120619583 2012-07-16 2012-06-16 12:00:00     6    Hollywood   
598768   120123632 2012-11-17 2012-11-16 20:30:00     1      Central   
631373   131411831 2013-04-26 2013-04-02 23:00:00    14      Pacific   
633764   131718469 2013-11-09 2013-11-09 18:30:00    17   Devonshire   
722953   131418620 2013-07-19 2013-07-19 19:40:00    14      Pacific   
755801   130510483 2013-05-26 2013-05-25 20:00:00     5       Harbor   
823129   141225513 2014-11-26 2014-11-26 11:00:00    12  77th Street   
830482   140606335 2014-02-12 2014-02-12 00:30:00     6    Hollywood   
841106   141308108 2014-03-16 2014-03-16 15:30:00    13       Newton   
847932   140105268 2014-01-18 2014-01-18 09:30:00     1      Central   
854169   141218155 2014-08-09 2014-07-30 12:00:00    12  77th Street   
860348   140308003 2014-03-13 2014-03-11 20:00:00     3    Southwest   
931959   141223962 2014-10-31 2014-10-30 15:00:00    12  77th Street   
973320   141314455 2014-07-12 2014-07-11 02:00:00    13       Newton   
1011274  151224700 2015-10-29 2015-10-28 12:00:00    12  77th Street   
1058874  151216330 2015-06-30 2015-06-26 21:30:00    12  77th Street   
1115808  151316498 2015-08-17 2015-08-14 15:00:00    13       Newton   
1254665  160217814 2016-09-27 2016-09-26 16:45:00     2      Rampart   
1279006  160327645 2016-12-20 2016-12-20 13:50:00     3    Southwest   
1359864  161922607 2016-11-30 2016-11-25 09:50:00    19      Mission   
1510762  170314115 2017-06-01 2017-06-01 00:30:00     3    Southwest   
1625775  171221278 2017-08-25 2017-07-29 14:00:00    12  77th Street   
1871854  181105575 2018-02-06 2018-02-05 17:20:00    11    Northeast   
1947458  191307542 2019-03-06 2019-03-06 16:45:00    13       Newton   
1959362  192117351 2019-10-12 2019-10-12 17:00:00    21      Topanga   
1968622  201604049 2020-01-02 2019-11-20 15:30:00    16     Foothill   
1973869  191421123 2019-09-29 2019-09-28 23:30:00    14      Pacific   
1975814  201604048 2020-01-02 2019-11-20 15:30:00    16     Foothill   

         Rpt Dist No  Part 1-2  Crm Cd  \
100040          1248         1     210   
151803          1764         2     653   
160732          1756         2     900   
219776          2139         1     331   
285526          1266         1     210   
400260          1243         2     920   
485891          1268         2     354   
563146           644         2     922   
598768           192         2     888   
631373          1494         1     440   
633764          1718         1     310   
722953          1441         1     210   
755801           507         1     440   
823129          1241         1     341   
830482           632         1     310   
841106          1347         1     510   
847932           157         1     310   
854169          1251         2     664   
860348           308         1     330   
931959          1239         2     920   
973320          1352         1     440   
1011274         1267         1     210   
1058874         1243         1     230   
1115808         1367         2     740   
1254665          247         2     664   
1279006          361         1     442   
1359864         1912         1     420   
1510762          337         2     740   
1625775         1245         1     440   
1871854         1183         2     745   
1947458         1307         1     510   
1959362         2145         1     440   
1968622         1622         2    

In [45]:
indexes_status = la_crimes.loc[(la_crimes['Status'] == '19')|
                              (la_crimes['Status'] == '13') | (la_crimes['Status'] == 'TH')|
                               (la_crimes['Status'].isna())
                              ]['Status'].index #finding the indexes of nan Status or status 19,13,TH
la_crimes.loc[indexes_status,'Status'] = 'CC'

* We consider that the value `UNK` for the column `Status Desc` means unknown
* Then we will change the Nan values for column `Status Desc` to `UNK`

In [46]:
indexes_status_desc = la_crimes.loc[la_crimes['Status Desc'].isna()]['Status'].index #finding the indexes of nan Status or status 19,13,TH
la_crimes.loc[indexes_status_desc,'Status Desc'] = 'UNK'

## Crm Cd columns
Also, reading the documentation of the columns `Crm Cd` indicates the crime committed. `Crm Cd 1` is the primary and most serious one. Crime Code 2, 3, and 4 are respectively less serious offenses
* As a result it doesn't make sense to only have a `Crm Cd 2` offense with no `Crm Cd 1` offense
* Thus we will update all crime reports with null `Crm Cd 1` and some `Crm Cd 2` or `Crm Cd 3` offenses so that in every case `Crm Cd 1` has a value.
* If there were 2 offenses then there will be only values for `Crm Cd 1` and `Crm Cd 2` and so on

In [47]:
indexes_crm1null = la_crimes.loc[(la_crimes ['Crm Cd 1'].isna()) ].index
la_crimes.loc[indexes_crm1null,:]

DR_NO  Date Rptd            DATE OCC  AREA  AREA NAME  \
55532    100707214 2010-03-14 2010-03-13 02:30:00     7   Wilshire   
288235   110310134 2011-04-08 2011-03-26 08:00:00     3  Southwest   
358506   110811926 2011-07-01 2011-07-01 20:11:00     8    West LA   
507665   120325216 2012-11-19 2012-11-19 19:30:00     3  Southwest   
1176227  150318476 2015-08-17 2015-08-16 12:00:00     3  Southwest   
1188315  150517852 2015-11-09 2015-10-09 18:00:00     5     Harbor   
1697697  181824031 2018-12-13 2018-12-13 19:40:00    18  Southeast   
1875523  181117551 2018-10-05 2018-10-05 09:00:00    11  Northeast   
1977968  191400827 2019-07-20 2019-07-20 09:45:00    14    Pacific   
2080625  190308122 2019-03-05 2019-03-05 21:30:00     3  Southwest   

         Rpt Dist No  Part 1-2  Crm Cd  \
55532            767         2     624   
288235           312         2     942   
358506           835         1     210   
507665           329         1     440   
1176227          363         1     761   
1188315          529         1     420   
1697697         1842         1     230   
1875523         1162         1     230   
1977968         1463         2     888   
2080625          395         1     331   

                                              Crm Cd Desc  \
55532                            BATTERY - SIMPLE ASSAULT   
288235                                            BRIBERY   
358506                                            ROBBERY   
507665                 THEFT PLAIN - PETTY ($950 & UNDER)   
1176227                                   BRANDISH WEAPON   
1188315   THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
1697697    ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
1875523    ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
1977968                                       TRESPASSING   
2080625  THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)   

                                         Mocodes  ...  Status   Status Desc  \
55532                        0344 0400 0416 1300  ...      IC   Invest Cont   
288235                                 1300 1402  ...      IC   Invest Cont   
358506                                   unknown  ...      IC   Invest Cont   
507665                                      0344  ...      IC   Invest Cont   
1176227  0913 0906 0334 0421 0319 0444 0432 1816  ...      AO   Adult Other   
1188315                      0344 1300 1606 0321  ...      IC   Invest Cont   
1697697                           0906 0450 1402  ...      IC   Invest Cont   
1875523                 0445 0432 1822 0443 1266  ...      AA  Adult Arrest   
1977968                                     1501  ...      IC   Invest Cont   
2080625                 0216 0344 1606 1822 1300  ...      IC   Invest Cont   

        Crm Cd 1  Crm Cd 2 Crm Cd 3  Crm Cd 4  \
55532        NaN     624.0      NaN       NaN   
288235       NaN     942.0     99.0       NaN   
358506       NaN     210.0      NaN       NaN   
507665       NaN     440.0      NaN       NaN   
1176227      NaN     761.0     93.0       NaN   
1188315      NaN     420.0      NaN       NaN   
1697697      NaN     230.0      NaN       NaN   
1875523      NaN     230.0     93.0       NaN   
1977968      NaN     888.0      NaN       NaN   
2080625      NaN     331.0      NaN       NaN   

                                         LOCATION  \
55532             PICO                         BL   
288235    5100    ROSELAND                     ST   
358506   11000    SANTA MONICA                 BL   
507665     500 W  27TH                         ST   
1176227   4100    PALMWOOD                     DR   
1188315                              200    BERTH   
1697697    400 W  109TH                        ST   
1875523   1300 N  VERMONT                      AV   
1977968   5300    ALLA                         RD   
2080625           39TH                         ST   

                            Cross Street      LAT       LON  
55532    NORTON                       AV  34.0476 -

In [48]:
la_crimes.loc[indexes_crm1null,'Crm Cd 1'] = la_crimes.loc[indexes_crm1null,'Crm Cd 2']
la_crimes.loc[indexes_crm1null,'Crm Cd 2'] = la_crimes.loc[indexes_crm1null,'Crm Cd 3']
la_crimes.loc[indexes_crm1null,'Crm Cd 3'] = la_crimes.loc[indexes_crm1null,'Crm Cd 4']

We fill up all the null values for `Crm Cd 2` and `Crm Cd 3` and `Crm Cd 4` with -1 (meaning none is found) because those columns consist of float datatype

In [49]:
la_crimes ['Crm Cd 2'] = la_crimes ['Crm Cd 2'].fillna(-1)
la_crimes ['Crm Cd 3'] = la_crimes ['Crm Cd 3'].fillna(-1)
la_crimes ['Crm Cd 4'] = la_crimes ['Crm Cd 4'].fillna(-1)

Lastly, we transform the columns `Crm Cd 1`, `Crm Cd 2`, `Crm Cd 3`, `Crm Cd 4` from float to integer

In [50]:
la_crimes['Crm Cd 1'] = la_crimes['Crm Cd 1'].astype(int)
la_crimes['Crm Cd 2'] = la_crimes['Crm Cd 2'].astype(int)
la_crimes['Crm Cd 3'] = la_crimes['Crm Cd 3'].astype(int)
la_crimes['Crm Cd 4'] = la_crimes['Crm Cd 4'].astype(int)

* We check if for every crime incident, the column `Crm Cd` and `Crm Cd 1` are the same.
* Some are not the same so we consider them mistaken because `Crm Cd 1` is the primary crime commited and `Crm Cd` describes the crime commited
* For those incidents we set `Crm Cd 1` to be equal to `Crm Cd`

* First case is that `Crm Cd` has the same value as `Crm Cd 2`
* Then we just have to swap `Crm Cd 1` and `Crm Cd 2`

In [51]:
indexes_crm_crm2 = la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 2']].index
la_crimes.loc[indexes_crm_crm2,'Crm Cd 2'] = la_crimes.loc[indexes_crm_crm2,'Crm Cd 1'].copy()
la_crimes.loc[indexes_crm_crm2,'Crm Cd 1'] = la_crimes.loc[indexes_crm_crm2,'Crm Cd'].copy()

* Second case is that `Crm Cd` has the same value as `Crm Cd 3`
* Then we just have to swap `Crm Cd 1` and `Crm Cd 3`

In [52]:
indexes_crm_crm3 = la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 3']].index
la_crimes.loc[indexes_crm_crm3,'Crm Cd 3'] = la_crimes.loc[indexes_crm_crm3,'Crm Cd 1'].copy()
la_crimes.loc[indexes_crm_crm3,'Crm Cd 1'] = la_crimes.loc[indexes_crm_crm3,'Crm Cd'].copy()

* Third case is that `Crm Cd` has the same value as `Crm Cd 4`
* However, there is no incident like that

In [53]:
la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 4']].index

Int64Index([], dtype='int64')

* Lastly, because `Crm Cd` and `Crm Cd 1` are the same now, we can drop `Crm Cd 1`

In [54]:
la_crimes = la_crimes.drop(['Crm Cd 1'], axis=1)

## LOCATION Column
* Some locations have a bunch of white spaces in between their words

In [55]:
la_crimes ['LOCATION'].value_counts()

6TH                          ST            4756
7TH                          ST            3774
9300    TAMPA                        AV    3658
6TH                                        3235
6600    TOPANGA CANYON               BL    3064
                                           ... 
6400    GLENALBYN                    DR       1
1300    FIDORA                       ST       1
600    FLOWER                       CT        1
1000    HOFFMAN                      AV       1
ELLITA                                        1
Name: LOCATION, Length: 75251, dtype: int64

* The unnecessary white spaces are removed from the middle, front and back of the words

In [56]:
 la_crimes['LOCATION'] = la_crimes['LOCATION'].str.replace(' +', ' ').str.strip()

## Cross Street column
* Moreover, we see that there are many null values for the column `Cross Street`
* We changed those to 'unknown'

In [57]:
indexes_null_cross = la_crimes.loc[la_crimes ['Cross Street'].isna()].index
la_crimes.loc[indexes_null_cross,'Cross Street'] = 'unknown'

* In addition, we see many cross streets have a bunch of white spaces in between the name and the street (AV/BL).

In [58]:
la_crimes['Cross Street'].value_counts()

unknown                               1759334
BROADWAY                                 6157
FIGUEROA                                 3801
VERMONT                      AV          3746
SAN PEDRO                                3659
                                       ...   
E  GENTRY                       AV          1
FARIAS                       AV             1
WESTPARK                     ST             1
SIERRA PASS                                 1
ELLITA                                      1
Name: Cross Street, Length: 12869, dtype: int64

* We are going to remove the unnecessary white spaces in the middle, front and back of the words 

In [59]:
la_crimes['Cross Street'] = la_crimes['Cross Street'].str.replace(' +', ' ').str.strip()

* Now we observe that for most crime incidents this column has the value `unknown`
* In detail that is true, for the $1.76$ million incidents out of the $2.1$ million incidents that there are
* Thus we decide to drop this column

In [60]:
la_crimes.loc[la_crimes['Cross Street'] == 'unknown']

DR_NO  Date Rptd            DATE OCC  AREA   AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13      Newton   
2         70309629 2010-08-09 2010-08-09 15:15:00    13      Newton   
5        100100506 2010-01-05 2010-01-04 16:50:00     1     Central   
7        100100509 2010-01-09 2010-01-08 21:00:00     1     Central   
8        100100510 2010-01-09 2010-01-09 02:30:00     1     Central   
...            ...        ...                 ...   ...         ...   
2114692  190906699 2019-02-24 2019-02-23 22:20:00     9    Van Nuys   
2114693  190506304 2019-02-22 2019-02-22 08:40:00     5      Harbor   
2114694  190608903 2019-03-28 2019-03-28 04:00:00     6   Hollywood   
2114697  191716777 2019-10-17 2019-10-16 18:00:00    17  Devonshire   
2114698  190805435 2019-02-01 2019-02-01 16:15:00     8     West LA   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
2               1324         2     946   
5                162         1     442   
7                157         1     230   
8                171         1     230   
...              ...       ...     ...   
2114692          904         1     210   
2114693          569         2     627   
2114694          644         1     648   
2114697         1795         1     420   
2114698          852         1     330   

                                             Crm Cd Desc  \
0                               VIOLATION OF COURT ORDER   
2                              OTHER MISCELLANEOUS CRIME   
5               SHOPLIFTING - PETTY THEFT ($950 & UNDER)   
7         ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
8         ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
...                                                  ...   
2114692                                          ROBBERY   
2114693          CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT   
2114694                                            ARSON   
2114697  THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
2114698                            BURGLARY FROM VEHICLE   

                                         Mocodes  ...  \
0                                 0913 1814 2000  ...   
2                                           0344  ...   
5                                      0344 1402  ...   
7                                           0416  ...   
8                                      0400 0416  ...   
...                                          ...  ...   
2114692  0344 0302 0334 0355 1310 1420 1822 0354  ...   
2114693                      0443 0419 0416 1259  ...   
2114694                                0601 1501  ...   
2114697                                  unknown  ...   
2114698       1302 1609 0358 1307 0344 0377 0321  ...   

                                            Weapon Desc Status   Status Desc  \
0                                               unknown     AA  Adult Arrest   
2                                               unknown     IC   Invest Cont   
5                                               unknown     AA  Adult Arrest   
7                           UNKNOWN WEAPON/OTHER WEAPON     AA  Adult Arrest   
8        STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     IC   Invest Cont   
...                                                 ...    ...           ...   
2114692                                   OTHER FIREARM     IC   Invest Cont   
2114693  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     AO   Adult Other   
2114694                                            FIRE     IC   Invest Cont   
2114697                                         unknown     IC   Invest Cont   
2114698                                         unknown     IC   Invest Cont   

         Crm Cd 2 Crm Cd 3  Crm Cd 4           LOCATION Cross Street      LAT  \
0              -1       -1        -1      300 E GAGE AV      unknown  33.9825   
2              -1       -1        -1     1300 E 21ST ST      unknown  34.0224   
5              -1       -1        -1       70

In [61]:
la_crimes = la_crimes.drop(['Cross Street'], axis=1)

## LONGTITUDE-LATITUDE
* We see that some crimes have values for longtitude and latitude equal to $0$
* Those values are considered missing because they correspond to the Gulf of Guinea.

In [62]:
la_crimes.loc[(la_crimes['LON'] == 0) & (la_crimes['LAT'] == 0) ]

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
49703    100618355 2010-07-14 2010-07-12 19:00:00     6    Hollywood   
49800    100618603 2010-07-19 2010-07-19 23:45:00     6    Hollywood   
60848    100718479 2010-11-29 2010-11-29 16:30:00     7     Wilshire   
84978    101016365 2010-09-09 2010-08-23 15:00:00    10  West Valley   
123985   101409719 2010-04-01 2010-03-30 21:00:00    14      Pacific   
...            ...        ...                 ...   ...          ...   
1982414  191915311 2019-08-22 2019-08-21 11:00:00    19      Mission   
1995724  191409480 2019-03-31 2019-03-31 03:00:00    14      Pacific   
2013881  190918499 2019-10-18 2019-10-15 16:30:00     9     Van Nuys   
2022655  191719423 2019-12-15 2019-12-13 11:00:00    17   Devonshire   
2078950  200114120 2020-06-29 2019-12-28 16:20:00     1      Central   

         Rpt Dist No  Part 1-2  Crm Cd  \
49703            665         1     330   
49800            665         2     740   
60848            709         1     230   
84978           1000         2     626   
123985          1412         1     510   
...              ...       ...     ...   
1982414         1900         1     236   
1995724         1400         2     624   
2013881          936         2     626   
2022655         1786         1     420   
2078950          162         2     354   

                                               Crm Cd Desc  \
49703                                BURGLARY FROM VEHICLE   
49800    VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...   
60848       ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
84978                    INTIMATE PARTNER - SIMPLE ASSAULT   
123985                                    VEHICLE - STOLEN   
...                                                    ...   
1982414              INTIMATE PARTNER - AGGRAVATED ASSAULT   
1995724                           BATTERY - SIMPLE ASSAULT   
2013881                  INTIMATE PARTNER - SIMPLE ASSAULT   
2022655    THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)   
2078950                                  THEFT OF IDENTITY   

                     Mocodes  ...  Weapon Used Cd  \
49703         0344 1300 1302  ...               0   
49800              0329 0906  ...               0   
60848              0416 1402  ...             122   
84978              0416 2000  ...               0   
123985               unknown  ...               0   
...                      ...  ...             ...   
1982414  0400 0448 2000 0913  ...             400   
1995724            0400 0416  ...             400   
2013881       1814 2000 0448  ...             400   
2022655              unknown  ...               0   
2078950                 1501  ...               0   

                                            Weapon Desc Status   Status Desc  \
49703                                           unknown     IC   Invest Cont   
49800                                           unknown     AA  Adult Arrest   
60848     HECKLER & KOCH 93 SEMIAUTOMATIC ASSAULT RIFLE     IC   Invest Cont   
84978                                           unknown     IC   Invest Cont   
123985                                          unknown     IC   Invest Cont   
...                                                 ...    ...           ...   
1982414  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     AA  Adult Arrest   
1995724  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     IC   Invest Cont   
2013881  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     AA  Adult Arrest   
2022655                                         unknown     IC   Invest Cont   
2078950                                         unknown     IC   Invest Cont   

        Crm Cd 2  Crm Cd 3 Crm Cd 4          LOCATION  LAT  LON  
49703         -1        -1       -1  900 N CISTRUS AV  0.0  0.0  
49800        998        -1       -1    6300 WILLOUGBY  0.0  0.0  
60848        998        -1       -1            HARBOR  0.0  0.0  
84978         -1        -1       -1 

# SAVING THE CLEANED DATASET
* We save the cleaned dataset as a new csv file called `LA Crime_Cleared.csv`, since we are going to use it for analysis purposes after

In [63]:
la_crimes.to_csv('LA Crime_Cleared.csv', index=False)

# COLUMNS SEPARATING + DIMENSION TABLES

 We will organize the database based on a star schema. In the middle the fact table will contain the crime incidents. In addition, there will be several different dimensions which will relate to each particular incident. Specifically the dimensions will be the location, the date reported,the date occured, the suspect, the victim, the crime commited, the premises (meaning the type of structure, vehicle, or location where the crime took place), the weapon used, the status of the crime and the coordinate dimension

## Location dimension
* We start by creating the dimension of the location which will include the location, the district, the area code and the area name 
* Particularly we take the columns `LOCATION` `Rpt Dist No` and `AREA NAME` and clear the duplicate values
* The results are saved in a new dataframe called `location_dimension`

In [61]:
la_crimes[['LOCATION','Rpt Dist No','AREA','AREA NAME']].drop_duplicates()

LOCATION  Rpt Dist No  AREA  AREA NAME
0         300 E GAGE AV         1385    13     Newton
1          SEPULVEDA BL         1485    14    Pacific
2        1300 E 21ST ST         1324    13     Newton
3           CAHUENGA BL          646     6  Hollywood
4                8TH ST          176     1    Central
...                 ...          ...   ...        ...
2114446    8300 81ST ST         1483    14    Pacific
2114462    4500 ALAMEDA         1367    13     Newton
2114538      WASHINGTON         2069    20    Olympic
2114594         CRYSTAL         1133    11  Northeast
2114634  WESTERN CANYON         1161    11  Northeast

[123751 rows x 4 columns]

* We see that each district number (`Rpt Dist No`) corresponds to one area name
* That means if we just know the district where a crime happened, we can find out the area where it took place because every district belongs to an area
* However, each location can correspond to different districts and areas because of the likelihood of using the same names

In [62]:
la_crimes[['Rpt Dist No','AREA NAME']].drop_duplicates()

Rpt Dist No    AREA NAME
0               1385       Newton
1               1485      Pacific
2               1324       Newton
3                646    Hollywood
4                176      Central
...              ...          ...
1967201         1086  West Valley
2057309         1686     Foothill
2077942         1784   Devonshire
2089763         1647     Foothill
2095715          693    Hollywood

[1303 rows x 2 columns]

In [63]:
location_dimension = la_crimes[['LOCATION','Rpt Dist No','AREA NAME']].drop_duplicates()
location_dimension = location_dimension.reset_index(drop = True)
location_dimension['LOCATION_CD'] = location_dimension.index
location_dimension

LOCATION  Rpt Dist No  AREA NAME  LOCATION_CD
0        300 E GAGE AV         1385     Newton            0
1         SEPULVEDA BL         1485    Pacific            1
2       1300 E 21ST ST         1324     Newton            2
3          CAHUENGA BL          646  Hollywood            3
4               8TH ST          176    Central            4
...                ...          ...        ...          ...
123746    8300 81ST ST         1483    Pacific       123746
123747    4500 ALAMEDA         1367     Newton       123747
123748      WASHINGTON         2069    Olympic       123748
123749         CRYSTAL         1133  Northeast       123749
123750  WESTERN CANYON         1161  Northeast       123750

[123751 rows x 4 columns]

* We will use the index column of the new dataframe `location_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `LOCATION` `Rpt Dist No` `ΑREA` and `AREA NAME` are no longer needed in the dataframe `la_crimes` as we have the mapping from the column `LOCATION_CD` to the dimension table `location_dimension`

In [64]:
la_crimes = pd.merge(la_crimes, 
               location_dimension, 
               on=['LOCATION','Rpt Dist No','AREA NAME'],
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA  AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13     Newton   
1        101304962 2010-01-14 2010-01-14 20:45:00    13     Newton   
2        101304963 2010-01-14 2010-01-14 20:45:00    13     Newton   
3        101305011 2010-01-15 2010-01-15 16:15:00    13     Newton   
4        101307014 2010-02-16 2010-02-16 21:00:00    13     Newton   
...            ...        ...                 ...   ...        ...   
2114694  191416200 2019-07-16 2019-06-24 12:00:00    14    Pacific   
2114695  191308117 2019-03-16 2019-03-16 14:30:00    13     Newton   
2114696  192019775 2019-11-09 2019-11-09 23:00:00    20    Olympic   
2114697  191114224 2019-07-27 2019-07-22 04:30:00    11  Northeast   
2114698  191106804 2019-02-27 2019-02-26 17:00:00    11  Northeast   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1385         2     946   
2               1385         2     946   
3               1385         2     946   
4               1385         1     331   
...              ...       ...     ...   
2114694         1483         2     354   
2114695         1367         1     350   
2114696         2069         2     946   
2114697         1133         1     230   
2114698         1161         1     330   

                                              Crm Cd Desc  \
0                                VIOLATION OF COURT ORDER   
1                               OTHER MISCELLANEOUS CRIME   
2                               OTHER MISCELLANEOUS CRIME   
3                               OTHER MISCELLANEOUS CRIME   
4        THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)   
...                                                   ...   
2114694                                 THEFT OF IDENTITY   
2114695                                     THEFT, PERSON   
2114696                         OTHER MISCELLANEOUS CRIME   
2114697    ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
2114698                             BURGLARY FROM VEHICLE   

                                    Mocodes  ...  \
0                            0913 1814 2000  ...   
1                                 0421 0906  ...   
2                                      0421  ...   
3                            0334 0432 0913  ...   
4                                 0344 1606  ...   
...                                     ...  ...   
2114694                                1822  ...   
2114695  0344 0602 0554 0907 1251 1259 0416  ...   
2114696                           1300 1501  ...   
2114697                           0416 0400  ...   
2114698                                1307  ...   

                                            Weapon Desc Status   Status Desc  \
0                                               unknown     AA  Adult Arrest   
1                                               unknown     AO   Adult Other   
2                                         VERBAL THREAT     IC   Invest Cont   
3                                               unknown     AO   Adult Other   
4                                               unknown     IC   Invest Cont   
...                                                 ...    ...           ...   
2114694                                         unknown     IC   Invest Cont   
2114695  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     IC   Invest Cont   
2114696                                         unknown     IC   Invest Cont   
2114697  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)     IC   Invest Cont   
2114698                                         unknown     IC   Invest Cont   

         Crm Cd 2 Crm Cd 3  Crm Cd 4        LOCATION      LAT       LON  \
0              -1       -1        -1   300 E GAGE AV  33.9825 -118.2695   
1              -1       -1        -1   300 E GAGE AV  33.9825 -118.2695   
2              -1       -1        -1   300 E GAGE AV  33.9825 -118.2695   
3              -1       -1        -1   300 E GAG

## Date reported dimension
* Next up we create the dimension of the date that the crime was reported
* Particularly we take the column `Date Rptd` and clear the duplicate values
* The results are saved in a new dataframe called `date_rptd_dimension`
* We also save the index of the new dataframe as `DATE_RPTD_CD`

In [65]:
date_rptd_dimension = la_crimes['Date Rptd'].drop_duplicates()
date_rptd_dimension = pd.DataFrame({'Date Rptd':date_rptd_dimension})
date_rptd_dimension = date_rptd_dimension.reset_index(drop = True)
date_rptd_dimension['DATE_RPTD_CD'] = date_rptd_dimension.index
date_rptd_dimension

Date Rptd  DATE_RPTD_CD
0    2010-02-20             0
1    2010-01-14             1
2    2010-01-15             2
3    2010-02-16             3
4    2010-04-03             4
...         ...           ...
3825 2020-03-21          3825
3826 2020-06-01          3826
3827 2020-06-07          3827
3828 2020-06-21          3828
3829 2020-06-28          3829

[3830 rows x 2 columns]

* We will use the index column of the new dataframe `date_rptd_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `Date Rptd` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `DATE_RPTD_CD` to the dimension table `date_rptd_dimension`

In [66]:
la_crimes = pd.merge(la_crimes, 
               date_rptd_dimension, 
               on='Date Rptd',
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13       Newton   
1        101307354 2010-02-20 2010-02-20 14:35:00    13       Newton   
2        101307355 2010-02-20 2010-02-20 13:50:00    13       Newton   
3        100106680 2010-02-20 2010-02-20 17:00:00     1      Central   
4        100106675 2010-02-20 2010-02-20 11:00:00     1      Central   
...            ...        ...                 ...   ...          ...   
2114694  201510987 2020-06-07 2013-01-01 12:00:00    15  N Hollywood   
2114695  201110178 2020-06-07 2019-08-16 13:30:00    11    Northeast   
2114696  201609806 2020-06-07 2012-06-01 12:00:00    16     Foothill   
2114697  201511599 2020-06-21 2019-05-23 14:00:00    15  N Hollywood   
2114698  201110925 2020-06-28 2019-05-25 14:05:00    11    Northeast   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1385         2     930   
2               1385         2     900   
3                152         2     930   
4                192         2     626   
...              ...       ...     ...   
2114694         1547         2     956   
2114695         1172         2     354   
2114696         1663         2     860   
2114697         1585         1     341   
2114698         1169         2     354   

                                               Crm Cd Desc         Mocodes  \
0                                 VIOLATION OF COURT ORDER  0913 1814 2000   
1                   CRIMINAL THREATS - NO WEAPON DISPLAYED  0371 0421 0906   
2                                 VIOLATION OF COURT ORDER  0913 1814 2000   
3                   CRIMINAL THREATS - NO WEAPON DISPLAYED       0421 0903   
4                        INTIMATE PARTNER - SIMPLE ASSAULT  0416 1241 2000   
...                                                    ...             ...   
2114694            LETTERS, LEWD  -  TELEPHONE CALLS, LEWD            1501   
2114695                                  THEFT OF IDENTITY            0377   
2114696                        BATTERY WITH SEXUAL CONTACT  1822 0522 1258   
2114697  THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...            0432   
2114698                                  THEFT OF IDENTITY       1822 1501   

         ...  Status   Status Desc Crm Cd 2  Crm Cd 3 Crm Cd 4  \
0        ...      AA  Adult Arrest       -1        -1       -1   
1        ...      AA  Adult Arrest       -1        -1       -1   
2        ...      IC   Invest Cont       -1        -1       -1   
3        ...      AA  Adult Arrest       -1        -1       -1   
4        ...      AO   Adult Other       -1        -1       -1   
...      ...     ...           ...      ...       ...      ...   
2114694  ...      IC   Invest Cont       -1        -1       -1   
2114695  ...      IC   Invest Cont       -1        -1       -1   
2114696  ...      IC   Invest Cont       -1        -1       -1   
2114697  ...      IC   Invest Cont       -1        -1       -1   
2114698  ...      IC   Invest Cont       -1        -1       -1   

                   LOCATION      LAT       LON LOCATION_CD  DATE_RPTD_CD  
0             300 E GAGE AV  33.9825 -118.2695           0             0  
1             300 E GAGE AV  33.9825 -118.2695           0             0  
2             300 E GAGE AV  33.9825 -118.2695           0             0  
3              600 W 5TH ST  34.0502 -118.2540          13             0  
4          1300 S FLOWER ST  34.0401 -118.2668          16             0  
...                     ...      ...       ...         ...           ...  
2114694         MAGNOLIA BL  34.1649 -118.3790       59455          3827  
2114695    1500 WATERLOO ST  34.0834 -118.2643       70760          3827  
2114696  12100 ALLEGHENY ST  34.2347 -118.3965       80080          3827  
2114697  10900 BLUFFSIDE DR  34.1414 -118.3686       38251          3828  
2114698        300 S AVE 56  34.1040 -118.1971      120931          3829  

[2114699 row

## Date occurred dimension
* Next up we create the dimension of the date that the crime happened
* Particularly we take the column `DATE OCC` and clear the duplicate values
* The results are saved in a new dataframe called `date_occ_dimension`
* We also save the index of the new dataframe as `DATE_OCC_CD`

In [67]:
date_occ_dimension = la_crimes['DATE OCC'].drop_duplicates()
date_occ_dimension = pd.DataFrame({'DATE OCC':date_occ_dimension})
date_occ_dimension = date_occ_dimension.reset_index(drop = True)
date_occ_dimension['DATE_OCC_CD'] = date_occ_dimension.index
date_occ_dimension

DATE OCC  DATE_OCC_CD
0      2010-02-20 13:50:00            0
1      2010-02-20 14:35:00            1
2      2010-02-20 17:00:00            2
3      2010-02-20 11:00:00            3
4      2010-02-16 08:15:00            4
...                    ...          ...
626060 2019-12-01 06:15:00       626060
626061 2013-12-01 12:55:00       626061
626062 2019-05-12 05:36:00       626062
626063 2015-06-07 04:20:00       626063
626064 2019-07-21 08:26:00       626064

[626065 rows x 2 columns]

* We will use the index column of the new dataframe `date_occ_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `DATE OCC` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `DATE_OCC_CD` to the dimension table `date_occ_dimension`

In [68]:
la_crimes = pd.merge(la_crimes, 
               date_occ_dimension, 
               on='DATE OCC',
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13    Newton   
1        101307355 2010-02-20 2010-02-20 13:50:00    13    Newton   
2        100906831 2010-02-20 2010-02-20 13:50:00     9  Van Nuys   
3        101307354 2010-02-20 2010-02-20 14:35:00    13    Newton   
4        100106680 2010-02-20 2010-02-20 17:00:00     1   Central   
...            ...        ...                 ...   ...       ...   
2114694  201411262 2020-05-22 2019-12-01 06:15:00    14   Pacific   
2114695  201609504 2020-05-22 2013-12-01 12:55:00    16  Foothill   
2114696  202007233 2020-03-07 2019-05-12 05:36:00    20   Olympic   
2114697  201412023 2020-06-07 2015-06-07 04:20:00    14   Pacific   
2114698  201411978 2020-06-07 2019-07-21 08:26:00    14   Pacific   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1385         2     900   
2                943         2     928   
3               1385         2     930   
4                152         2     930   
...              ...       ...     ...   
2114694         1431         2     745   
2114695         1675         2     354   
2114696         2015         2     354   
2114697         1434         1     121   
2114698         1465         2     956   

                                      Crm Cd Desc         Mocodes  ...  \
0                        VIOLATION OF COURT ORDER  0913 1814 2000  ...   
1                        VIOLATION OF COURT ORDER  0913 1814 2000  ...   
2                 THREATENING PHONE CALLS/LETTERS            0421  ...   
3          CRIMINAL THREATS - NO WEAPON DISPLAYED  0371 0421 0906  ...   
4          CRIMINAL THREATS - NO WEAPON DISPLAYED       0421 0903  ...   
...                                           ...             ...  ...   
2114694  VANDALISM - MISDEAMEANOR ($399 OR UNDER)            0329  ...   
2114695                         THEFT OF IDENTITY  1822 0377 0930  ...   
2114696                         THEFT OF IDENTITY            0928  ...   
2114697                            RAPE, FORCIBLE            0527  ...   
2114698   LETTERS, LEWD  -  TELEPHONE CALLS, LEWD            1501  ...   

          Status Desc Crm Cd 2 Crm Cd 3  Crm Cd 4             LOCATION  \
0        Adult Arrest       -1       -1        -1        300 E GAGE AV   
1         Invest Cont       -1       -1        -1        300 E GAGE AV   
2         Invest Cont       -1       -1        -1      14100 OXNARD ST   
3        Adult Arrest       -1       -1        -1        300 E GAGE AV   
4        Adult Arrest       -1       -1        -1         600 W 5TH ST   
...               ...      ...      ...       ...                  ...   
2114694   Invest Cont       -1       -1        -1            VENICE BL   
2114695   Invest Cont       -1       -1        -1      8700 HERRICK AV   
2114696   Invest Cont       -1       -1        -1  100 N ALEXANDRIA AV   
2114697   Invest Cont       -1       -1        -1    3600 BEETHOVEN ST   
2114698   Invest Cont       -1       -1        -1    12000 JUNIETTE ST   

             LAT       LON LOCATION_CD DATE_RPTD_CD  DATE_OCC_CD  
0        33.9825 -118.2695           0            0            0  
1        33.9825 -118.2695           0            0            0  
2        34.1794 -118.4400       20593            0            0  
3        33.9825 -118.2695           0            0            1  
4        34.0502 -118.2540          13            0            2  
...          ...       ...         ...          ...          ...  
2114694  33.9880 -118.4703       36701         3817       626060  
2114695  34.2279 -118.3692       41864         3817       626061  
2114696  34.0728 -118.2984       53958         3819       626062  
2114697  34.0045 -118.4448       37564         3827       626063  
2114698  33.9845 -118.4059       37835         3827       626064  

[2114699 rows x 28 columns]

## Suspect dimension 
* Now we create the dimension of the suspect. Mocodes are activities associated with the suspect in commission of the crime
* Particularly we take the column `Mocodes` and clear the duplicate values
* The results are saved in a new dataframe called `suspect_dimension`
* We also save the index of the new dataframe as `MOCODES_CD`

In [69]:
suspect_dimension = la_crimes['Mocodes'].drop_duplicates()
suspect_dimension = pd.DataFrame({'Mocodes':suspect_dimension})
suspect_dimension = suspect_dimension.reset_index(drop = True)
suspect_dimension['MOCODE_CD'] = suspect_dimension.index
suspect_dimension

Mocodes  MOCODE_CD
0                           0913 1814 2000          0
1                                     0421          1
2                           0371 0421 0906          2
3                                0421 0903          3
4                                     0344          4
...                                    ...        ...
483757            0344 1236 0913 1803 0601     483757
483758  1820 1259 1251 1815 0913 0421 0356     483758
483759                 0522 1251 1822 1259     483759
483760       0416 0444 0446 1402 1814 2000     483760
483761            0500 0519 0906 0913 1202     483761

[483762 rows x 2 columns]

* We will use the index column of the new dataframe `suspect_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `Mocodes` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `MOCODE_CD` to the dimension table `suspect_dimension`

In [70]:
la_crimes = pd.merge(la_crimes, 
               suspect_dimension, 
               on='Mocodes',
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA    AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13       Newton   
1        101307355 2010-02-20 2010-02-20 13:50:00    13       Newton   
2        100806537 2010-02-28 2010-02-20 14:45:00     8      West LA   
3        101300529 2010-01-15 2010-01-15 22:00:00    13       Newton   
4        111314249 2011-06-10 2010-09-01 08:00:00    13       Newton   
...            ...        ...                 ...   ...          ...   
2114694  201005549 2020-01-31 2019-12-20 11:35:00    10  West Valley   
2114695  201005547 2020-01-31 2019-12-20 11:35:00    10  West Valley   
2114696  202005463 2020-01-31 2018-08-16 20:18:00    20      Olympic   
2114697  202007944 2020-03-26 2019-03-26 05:10:00    20      Olympic   
2114698  201210693 2020-04-10 2019-11-01 18:06:00    12  77th Street   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1385         2     900   
2                885         2     624   
3               1371         1     310   
4               1372         2     900   
...              ...       ...     ...   
2114694         1037         2     930   
2114695         1037         2     930   
2114696         2056         2     860   
2114697         2013         2     626   
2114698         1215         1     821   

                                               Crm Cd Desc  \
0                                 VIOLATION OF COURT ORDER   
1                                 VIOLATION OF COURT ORDER   
2                                 BATTERY - SIMPLE ASSAULT   
3                                                 BURGLARY   
4                                 VIOLATION OF COURT ORDER   
...                                                    ...   
2114694             CRIMINAL THREATS - NO WEAPON DISPLAYED   
2114695             CRIMINAL THREATS - NO WEAPON DISPLAYED   
2114696                        BATTERY WITH SEXUAL CONTACT   
2114697                  INTIMATE PARTNER - SIMPLE ASSAULT   
2114698  SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO...   

                                    Mocodes  ...  Crm Cd 2 Crm Cd 3 Crm Cd 4  \
0                            0913 1814 2000  ...        -1       -1       -1   
1                            0913 1814 2000  ...        -1       -1       -1   
2                            0913 1814 2000  ...        -1       -1       -1   
3                            0913 1814 2000  ...        -1       -1       -1   
4                            0913 1814 2000  ...        -1       -1       -1   
...                                     ...  ...       ...      ...      ...   
2114694  1820 1259 1251 1815 0913 0421 0356  ...        -1       -1       -1   
2114695  1820 1259 1251 1815 0913 0421 0356  ...        -1       -1       -1   
2114696                 0522 1251 1822 1259  ...        -1       -1       -1   
2114697       0416 0444 0446 1402 1814 2000  ...       998       -1       -1   
2114698            0500 0519 0906 0913 1202  ...        -1       -1       -1   

                  LOCATION      LAT       LON LOCATION_CD DATE_RPTD_CD  \
0            300 E GAGE AV  33.9825 -118.2695           0            0   
1            300 E GAGE AV  33.9825 -118.2695           0            0   
2         2900 OVERLAND AV  34.0340 -118.4188       18018         3469   
3            100 W 57TH ST  33.9906 -118.2739       31858            2   
4            300 E 56TH ST  33.9916 -118.2701       31841         2159   
...                    ...      ...       ...         ...          ...   
2114694     6600 BALBOA BL  34.1895 -118.5100       24387         3796   
2114695     6600 BALBOA BL  34.1895 -118.5100       24387         3796   
2114696  1100 S VERMONT AV  34.0509 -118.2937       54178         3796   
2114697    100 S OXFORD AV  34.0737 -118.3079       53483         3805   
2114698     1000 W 58TH ST  33.9897 -118.2915       28892         3808   

        DATE_OCC_CD  MOCODE_CD  
0              

## Victim dimension
* Next up the dimension of the vicitm will be made
* Particularly we take the columns `Vict Sex` and `Vict Descent`  and clear the duplicate values
* The results are saved in a new dataframe called `victim_dimension`

In [71]:
victim_dimension = la_crimes[['Vict Sex','Vict Descent']].drop_duplicates()
victim_dimension = victim_dimension.reset_index(drop=True)
victim_dimension['VICTIM_CD'] = victim_dimension.index
victim_dimension

Vict Sex Vict Descent  VICTIM_CD
0         M            H          0
1         F            B          1
2         M            B          2
3         F            H          3
4         M            W          4
5         F            W          5
6         F            O          6
7         M            O          7
8         M            K          8
9         F            A          9
10        M            A         10
11        M            X         11
12        X            X         12
13        M            F         13
14        M            C         14
15        F            X         15
16        M            U         16
17        M            I         17
18        F            U         18
19        F            I         19
20        X            W         20
21        F            F         21
22        F            K         22
23        F            J         23
24        F            C         24
25        M            J         25
26        M            Z         26
27        F            V         27
28        M            V         28
29        F            P         29
30        F            L         30
31        X            H         31
32        M            P         32
33        F            G         33
34        X            F         34
35        M            S         35
36        M            D         36
37        M            L         37
38        F            S         38
39        F            Z         39
40        X            O         40
41        F            D         41
42        X            B         42
43        M            G         43
44        X            C         44
45        X            A         45
46        X            K         46
47        X            G         47

* We will use the index column of the new dataframe `victim_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `Vict Sex` `Vict Descent`is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `VICTIM_CD` to the dimension table `victim_dimension`

In [72]:
la_crimes = pd.merge(la_crimes, 
               victim_dimension, 
               on=['Vict Sex','Vict Descent'],
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA  AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13     Newton   
1        111315925 2011-07-03 2011-06-27 13:00:00    13     Newton   
2        152014532 2015-08-10 2015-08-07 15:30:00    20    Olympic   
3        152010566 2015-05-15 2015-05-15 23:00:00    20    Olympic   
4        100906832 2010-02-20 2010-02-20 14:00:00     9   Van Nuys   
...            ...        ...                 ...   ...        ...   
2114694  162008634 2016-04-05 2016-04-04 18:00:00    20    Olympic   
2114695  171119454 2017-11-14 2017-11-12 11:00:00    11  Northeast   
2114696  180320562 2018-09-06 2018-09-06 07:00:00     3  Southwest   
2114697  190516898 2019-10-15 2019-10-14 18:00:00     5     Harbor   
2114698  180129379 2018-12-02 2018-12-02 17:50:00     1    Central   

         Rpt Dist No  Part 1-2  Crm Cd                         Crm Cd Desc  \
0               1385         2     900            VIOLATION OF COURT ORDER   
1               1352         2     900            VIOLATION OF COURT ORDER   
2               2069         2     901      VIOLATION OF RESTRAINING ORDER   
3               2069         2     901      VIOLATION OF RESTRAINING ORDER   
4                943         2     928     THREATENING PHONE CALLS/LETTERS   
...              ...       ...     ...                                 ...   
2114694         2041         1     330               BURGLARY FROM VEHICLE   
2114695         1102         2     624            BATTERY - SIMPLE ASSAULT   
2114696          325         1     440  THEFT PLAIN - PETTY ($950 & UNDER)   
2114697          526         1     330               BURGLARY FROM VEHICLE   
2114698          122         2     624            BATTERY - SIMPLE ASSAULT   

                          Mocodes  ...  Crm Cd 3 Crm Cd 4  \
0                  0913 1814 2000  ...        -1       -1   
1                  0913 1814 2000  ...        -1       -1   
2                  0913 1814 2000  ...        -1       -1   
3                  0913 1814 2000  ...        -1       -1   
4                            0421  ...        -1       -1   
...                           ...  ...       ...      ...   
2114694            1612 0358 0344  ...        -1       -1   
2114695  0416 0432 1202 0448 0913  ...        -1       -1   
2114696                 0344 0354  ...        -1       -1   
2114697       0321 0377 1300 0344  ...        -1       -1   
2114698            1266 0917 0416  ...        -1       -1   

                       LOCATION      LAT       LON  LOCATION_CD DATE_RPTD_CD  \
0                 300 E GAGE AV  33.9825 -118.2695            0            0   
1                 400 E 47TH ST  34.0007 -118.2696        32620         2600   
2             1100 S VERMONT AV  34.0509 -118.2916        53352         1905   
3        1100 S WESTMORELAND AV  34.0508 -118.2891        53889         2294   
4               14100 OXNARD ST  34.1794 -118.4400        20593            0   
...                         ...      ...       ...          ...          ...   
2114694              900 3RD AV  34.0559 -118.3165        53659         1950   
2114695       5000 EDENHURST AV  34.1434 -118.2732        94835         1173   
2114696       2600 S BUDLONG AV  34.0328 -118.2959         3603         1280   
2114697           800 LAGOON AV  33.7797 -118.2675         9856         1377   
2114698           100 N HOPE ST  34.0564 -118.2498         1357         2649   

        DATE_OCC_CD MOCODE_CD  VICTIM_CD  
0                 0         0          0  
1            171067         0          0  
2            211794         0          0  
3            216664         0          0  
4                12         1          0  
...             ...       ...        ...  
2114694       88506     56943         46  
2114695       68947    110897         46  
2114696      286555      8480         47  
2114697       20609     45200         47  
2114698      508307    329940         47  

[2114699 rows x 30 colum

## Crime dimension
* Next up the dimension of the crime commited will be made
* Particularly we take the columns `Crm Cd` and `Crm Cd Desc` `Part 1-2` and clear the duplicate values
* After checking that each crime code corresponds to exactly one crime description and either part 1 or 2 crime type, there is no extra work to do.
* The results are saved in a new dataframe called `crime_dimension`

In [73]:
la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()['Crm Cd Desc'].value_counts() #check

DISHONEST EMPLOYEE - GRAND THEFT     1
KIDNAPPING                           1
FALSE IMPRISONMENT                   1
DOCUMENT WORTHLESS ($200 & UNDER)    1
LEWD CONDUCT                         1
                                    ..
DISTURBING THE PEACE                 1
PICKPOCKET, ATTEMPT                  1
ARSON                                1
THREATENING PHONE CALLS/LETTERS      1
DOCUMENT FORGERY / STOLEN FELONY     1
Name: Crm Cd Desc, Length: 142, dtype: int64

In [74]:
la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()['Crm Cd'].value_counts()#check

510    1
352    1
905    1
432    1
870    1
      ..
951    1
440    1
441    1
442    1
434    1
Name: Crm Cd, Length: 142, dtype: int64

In [75]:
crime_dimension = la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()

* Moreover, we add the crime code `Crm Cd` $-1$ as a code for no crime . This will be used for the the columns `Crm Cd 2`, `Crm Cd 3` and `Crm Cd 4` (instead of them being null) because some crime incidents only include one primary crime

In [76]:
crime_dimension = crime_dimension.append({'Crm Cd':-1, 'Crm Cd Desc': 'NO CRIME',  'Part 1-2': np.nan}, ignore_index=True)

* Now we have to make sure that even the `Crm Cd 2` `Crm Cd 3` and `Crm Cd 4` codes are included in the crime dimension.
* For that we create a new dataframe that that includes all  the unique `Crm Cd 2` `Crm Cd 3` `Crm Cd 4` codes

In [77]:
crm2 = la_crimes['Crm Cd 2'].drop_duplicates()
crm3 = la_crimes['Crm Cd 3'].drop_duplicates()
crm4 = la_crimes['Crm Cd 4'].drop_duplicates()
crm2 = crm2.append(crm3, ignore_index=True).drop_duplicates()
crm2 = crm2.append(crm4, ignore_index=True).drop_duplicates()
crm2

0       -1
1      998
2      930
3      900
4      940
      ... 
142    438
143    652
144     99
145    978
146    884
Length: 147, dtype: int64

In [78]:
crm_sec_codes= pd.DataFrame()
crm_sec_codes['Crm Cd'] = crm2.copy()
crm_sec_codes

Crm Cd
0        -1
1       998
2       930
3       900
4       940
..      ...
142     438
143     652
144      99
145     978
146     884

[147 rows x 1 columns]

* Now we can merge the crimecode dimension with the additional codes
* For the codes that we do not have any crime description, we change their `Crm Cd Desc` to `'No information'`

In [79]:
crime_dimension = pd.merge(crime_dimension, crm_sec_codes, on='Crm Cd', how='outer')
crime_dimension['Crm Cd Desc'] = crime_dimension['Crm Cd Desc'].fillna('No information')
crime_dimension

Crm Cd                              Crm Cd Desc  Part 1-2
0       900                 VIOLATION OF COURT ORDER       2.0
1       901           VIOLATION OF RESTRAINING ORDER       2.0
2       928          THREATENING PHONE CALLS/LETTERS       2.0
3       930   CRIMINAL THREATS - NO WEAPON DISPLAYED       2.0
4       956  LETTERS, LEWD  -  TELEPHONE CALLS, LEWD       2.0
..      ...                                      ...       ...
161     972                           No information       NaN
162     953                           No information       NaN
163     945                           No information       NaN
164      99                           No information       NaN
165     978                           No information       NaN

[166 rows x 3 columns]

## Premises dimension
* Now the dimension of the premises of the crime will be made. In detail, those are the type of structure, vehicle, or location where the crime took place.
* Particularly we take the columns `Premis Cd` and `Premis Desc` and clear the duplicate values

In [80]:
la_crimes[['Premis Cd','Premis Desc']].drop_duplicates()['Premis Desc'].value_counts()

RETIRED (DUPLICATE) DO NOT USE THIS CODE    3
STAPLES CENTER *                            1
DRIVE THRU*                                 1
FURNITURE STORE                             1
MARKET                                      1
                                           ..
MTA - GREEN LINE - AVALON                   1
WEBSITE                                     1
MTA - GOLD LINE - HIGHLAND PARK             1
FOSTER HOME BOYS OR GIRLS*                  1
METROLINK TRAIN                             1
Name: Premis Desc, Length: 320, dtype: int64

* We see that different premises codes correspond to the premis desc `RETIRED (DUPLICATE) DO NOT USE THIS CODE`
* Because the premise description says not to be used, we will changed all those premises codes and descriptions to 0 and 'unknown' respectively

In [81]:
indexes_premis_retired = la_crimes.loc[la_crimes['Premis Desc']== 'RETIRED (DUPLICATE) DO NOT USE THIS CODE'].index
la_crimes.loc[indexes_premis_retired,'Premis Desc'] = 'unknown'
la_crimes.loc[indexes_premis_retired,'Premis Cd'] = 0

* Each premises code corresponds to exactly one premises description
* The results are saved in a new dataframe called `premises_dimension`

In [82]:
premises_dimension = la_crimes[['Premis Cd','Premis Desc']].drop_duplicates()

## Weapon dimension
Next up the dimension of the weapon used will be created
* Particularly we take the columns `Weapon Used Cd` and `Weapon Desc` and clear the duplicate values
* The results are saved in a new dataframe called `weapon_dimension`

In [83]:
la_crimes[['Weapon Used Cd','Weapon Desc']].drop_duplicates()['Weapon Desc'].value_counts()

unknown              2
AXE                  1
HAMMER               1
HAND GUN             1
PIPE/METAL PIPE      1
                    ..
FIXED OBJECT         1
ROPE/LIGATURE        1
PHYSICAL PRESENCE    1
KITCHEN KNIFE        1
BOARD                1
Name: Weapon Desc, Length: 80, dtype: int64

* We see that different weapon codes correspond to the weapon desc `unknown`
* Because we have used the weapon code $0$ for null values to describe incidents where we do not know the weapon, we will change the other weapon code ($222$) to 0 as well.

In [84]:
la_crimes.loc[la_crimes['Weapon Desc']== 'unknown']['Weapon Used Cd'].value_counts()

0      1404460
222          1
Name: Weapon Used Cd, dtype: int64

In [85]:
index_weaponcd222 = la_crimes.loc[la_crimes['Weapon Used Cd']== 222].index
la_crimes.loc[index_weaponcd222,'Weapon Desc'] = 0

* Each weapon code corresponds to exactly one weapon description
* The results are saved in a new dataframe called `weapon_dimension`

In [86]:
weapon_dimension = la_crimes[['Weapon Used Cd','Weapon Desc']].drop_duplicates()

## Status dimension
* Next up the dimension of the status of the crime is made
* Particularly we take the columns `Status` and `Status Desc` and clear the duplicate values
* After checking that each status code corresponds to exactly one status description, there is no extra work to do.
* The results are saved in a new dataframe called `status_dimension`

In [87]:
la_crimes[['Status','Status Desc']].drop_duplicates()

Status   Status Desc
0          AA  Adult Arrest
1          AO   Adult Other
4          IC   Invest Cont
172        JO     Juv Other
557        JA    Juv Arrest
158469     CC           UNK

In [88]:
status_dimension = la_crimes[['Status','Status Desc']].drop_duplicates()

## Coordinate dimension
* Next up is the dimension of the geographic coordinates of the incident
* Particularly we take the columns `LON` and `LAT` and clear the duplicate values
* The results are saved in a new dataframe called `coordinates_dimension`
* We save the index of the new dataframe as a new column `COORD_CD`

In [89]:
coordinates_dimension = la_crimes[['LON','LAT']].drop_duplicates()
coordinates_dimension = coordinates_dimension.reset_index(drop=True)
coordinates_dimension['COORD_CD'] = coordinates_dimension.index
coordinates_dimension

LON      LAT  COORD_CD
0     -118.2695  33.9825         0
1     -118.2696  34.0007         1
2     -118.2916  34.0509         2
3     -118.2891  34.0508         3
4     -118.4400  34.1794         4
...         ...      ...       ...
64482 -118.6071  34.2636     64482
64483 -118.6138  34.1848     64483
64484 -118.2894  33.9088     64484
64485 -118.4209  34.2301     64485
64486 -118.4210  34.0565     64486

[64487 rows x 3 columns]

* We will use the index column of the new dataframe `coordinates_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `LAT` and `LON` are no longer needed in the dataframe `la_crimes` as we have the mapping from the column `COORD_CD` to the dimension table `coordinates_dimension`

In [90]:
la_crimes = pd.merge(la_crimes, 
               coordinates_dimension, 
               on=['LON','LAT'],
               how='inner')
la_crimes

DR_NO  Date Rptd            DATE OCC  AREA   AREA NAME  \
0          1307355 2010-02-20 2010-02-20 13:50:00    13      Newton   
1        121321665 2012-10-27 2012-10-27 16:15:00    13      Newton   
2        171318398 2017-08-14 2017-08-11 06:00:00    13      Newton   
3        151312312 2015-05-30 2015-05-30 17:00:00    13      Newton   
4        121314885 2012-07-02 2012-07-02 13:40:00    13      Newton   
...            ...        ...                 ...   ...         ...   
2114694  131704213 2013-01-05 2013-01-05 18:30:00    17  Devonshire   
2114695  172109827 2017-04-25 2017-04-25 10:00:00    21     Topanga   
2114696  161807341 2016-03-08 2016-03-06 18:00:00    18   Southeast   
2114697  191917901 2019-10-14 2019-10-14 14:30:00    19     Mission   
2114698  110818769 2011-11-30 2011-11-30 08:20:00     8     West LA   

         Rpt Dist No  Part 1-2  Crm Cd  \
0               1385         2     900   
1               1385         1     440   
2               1395         1     331   
3               1395         1     440   
4               1383         2     624   
...              ...       ...     ...   
2114694         1721         1     310   
2114695         2155         2     662   
2114696         1871         2     940   
2114697         1988         1     230   
2114698          836         1     310   

                                              Crm Cd Desc  \
0                                VIOLATION OF COURT ORDER   
1                      THEFT PLAIN - PETTY ($950 & UNDER)   
2        THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER)   
3                      THEFT PLAIN - PETTY ($950 & UNDER)   
4                                BATTERY - SIMPLE ASSAULT   
...                                                   ...   
2114694                                          BURGLARY   
2114695                                BUNCO, GRAND THEFT   
2114696                                         EXTORTION   
2114697    ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT   
2114698                                          BURGLARY   

                                                   Mocodes  ...  Crm Cd 4  \
0                                           0913 1814 2000  ...        -1   
1                                                     0344  ...        -1   
2                                                     0344  ...        -1   
3                                                     0344  ...        -1   
4                                                     0416  ...        -1   
...                                                    ...  ...       ...   
2114694                                          1609 0344  ...        -1   
2114695                                     0800 1202 1822  ...        -1   
2114696            0443 0432 1906 1823 0513 0514 0520 0528  ...        -1   
2114697  0416 0913 1218 0447 0340 2004 0345 0334 0344 1223  ...        -1   
2114698                                          0344 1606  ...        -1   

                    LOCATION      LAT       LON LOCATION_CD  DATE_RPTD_CD  \
0              300 E GAGE AV  33.9825 -118.2695           0             0   
1              300 E GAGE AV  33.9825 -118.2695           0            15   
2              300 E GAGE AV  33.9825 -118.2695       68762           426   
3        6300 S SAN PEDRO ST  33.9825 -118.2695       33025           963   
4                  SAN PEDRO  33.9825 -118.2695       33504          3271   
...                      ...      ...       ...         ...           ...   
2114694    22000 LINFIELD AV  34.2636 -118.6071       93185           748   
2114695            SYLVAN ST  34.1848 -118.6138      113827          1170   
2114696         3700 S MENLO  33.9088 -118.2894      105196          1060   
2114697   13100 WENTWORTH ST  34.2301 -118.4209       51106          1390   
2114698   17000 LA GRANGE AV  34.0565 -118.4210       74660            10   

        DATE_OCC_CD MOCODE_CD VICTIM_CD  COORD_CD  
0                 0 

## Removing dimensions from the fact table-dataframe
* Now we can drop the extra columns that there are on our fact table 
* Particularly, many columns are dropped because we already made the dimensions dataframes which contain the codes and the description for each dimension
* We are going to keep the codes of the dimensions and whenever we need some extra information, we can look up the dimension tables

In [91]:
la_crimes = la_crimes.drop(['AREA','AREA NAME','LOCATION','Rpt Dist No','Date Rptd', 'Crm Cd Desc', 'Part 1-2',
                            'Premis Desc', 'DATE OCC', 'Mocodes','Vict Sex', 'Vict Descent',
                            'Weapon Desc', 'Status Desc','LON','LAT'], axis=1)
la_crimes

DR_NO  Crm Cd  Vict Age  Premis Cd  Weapon Used Cd Status  \
0          1307355     900        48        501               0     AA   
1        121321665     440        43        119               0     IC   
2        171318398     331        34        104               0     IC   
3        151312312     440        33        101               0     IC   
4        121314885     624        16        102             400     IC   
...            ...     ...       ...        ...             ...    ...   
2114694  131704213     310        43        501               0     IC   
2114695  172109827     662        60        101               0     IC   
2114696  161807341     940        22        501               0     IC   
2114697  191917901     230        37        102             311     IC   
2114698  110818769     310        49        501               0     IC   

         Crm Cd 2  Crm Cd 3  Crm Cd 4  LOCATION_CD  DATE_RPTD_CD  DATE_OCC_CD  \
0              -1        -1        -1            0             0            0   
1              -1        -1        -1            0            15         4221   
2              -1        -1        -1        68762           426        11219   
3              -1        -1        -1        33025           963       212429   
4              -1        -1        -1        33504          3271        42901   
...           ...       ...       ...          ...           ...          ...   
2114694        -1        -1        -1        93185           748       185385   
2114695        -1        -1        -1       113827          1170       120122   
2114696        -1        -1        -1       105196          1060       118406   
2114697        -1        -1        -1        51106          1390        20547   
2114698        -1        -1        -1        74660            10         2991   

         MOCODE_CD  VICTIM_CD  COORD_CD  
0                0          0         0  
1                4          0         0  
2                4          0         0  
3                4          0         0  
4               66          0         0  
...            ...        ...       ...  
2114694       5461         28     64482  
2114695      46817         28     64483  
2114696     169640         28     64484  
2114697      45099         31     64485  
2114698         59         34     64486  

[2114699 rows x 15 columns]

Renaming the coumns name for easier use

In [92]:
la_crimes.rename(columns={'Crm Cd':'CRM_CD',
                         'Vict Age':'VICT_AGE',
                         'Premis Cd':'PREMIS_CD','Weapon Used Cd':'WEAPON_CD',
                         'Status':'STATUS','Crm Cd 2':'CRM_CD2',
                         'Crm Cd 3':'CRM_CD3', 'Crm Cd 4':'CRM_CD4'},
                         inplace = True)

In [93]:
location_dimension.rename(columns={'Rpt Dist No': 'RPT_DIST_NO','AREA NAME':'AREA_NAME','LOCATION':'LOCATION_NAME'}
                          ,inplace = True)
crime_dimension.rename(columns={'Crm Cd': 'CRM_CD','Crm Cd Desc':'CRM_DESCR','Part 1-2':'PART_1_2'},inplace = True)
premises_dimension.rename(columns={'Premis Cd': 'PREMIS_CD','Premis Desc':'PREMIS_DESCR'},inplace = True)
weapon_dimension.rename(columns={'Weapon Used Cd':'WEAPON_CD', 'Weapon Desc':'WEAPON_DESCR'},inplace = True)
status_dimension.rename(columns={'Status': 'STATUS','Status Desc':'STATUS_DESCR'},inplace = True)
date_occ_dimension.rename(columns={'DATE OCC': 'DATE_OCC'},inplace = True)
date_rptd_dimension.rename(columns={'Date Rptd': 'DATE_RPTD'},inplace = True)
victim_dimension.rename(columns={'Vict Sex': 'VICT_SEX','Vict Descent':'VICT_DESCENT'},inplace = True)
suspect_dimension.rename(columns={'Mocodes': 'MOCODES'},inplace = True)

Saving the fact table(dataframe) and dimension tables(dataframes) as  csv documents in order to insert them into the database (not necessary for data analysis)

la_crimes.to_csv('la_crimes_cleared.csv',index=False)
location_dimension.to_csv('location_dimension.csv',index=False)
crime_dimension.to_csv('crime_dimension.csv',index=False)
premises_dimension.to_csv('premises_dimension.csv',index=False)
weapon_dimension.to_csv('weapon_dimension.csv',index=False)
status_dimension.to_csv('status_dimension.csv',index=False)
date_occ_dimension.to_csv('date_occ_dimension.csv',index=False)
date_rptd_dimension.to_csv('date_rptd_dimension.csv',index=False)
victim_dimension.to_csv('victim_dimension.csv',index=False)
suspect_dimension.to_csv('suspect_dimension.csv',index=False)
coordinates_dimension.to_csv('coordinates_dimension.csv',index=False)